In [13]:
import cv2
import os
from ultralytics import YOLO

# Directory to save frames
SAVE_DIR = "E:\\swift\\ultralytics\\results\\frames"
os.makedirs(SAVE_DIR, exist_ok=True)

In [14]:
# Load YOLO model
model = YOLO("Swift.pt")  # pretrained YOLO model

# Class costs
class_costs = {
    "biscuit": 20,
    "chips": 10,
    "chocolate": 40,
    "juice": 30,
    "milk": 50,
    "paste": 20,
    "soap": 50,
}

In [15]:
# Initialize video capture
#cap = cv2.VideoCapture("http://192.168.19.156:4747/video")
cap = cv2.VideoCapture("http://192.168.132.176:4747/video")

#cap = cv2.VideoCapture(0)

print("Press 's' to save frame, 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    cv2.imshow("Frame", frame)

    # Capture key press
    key = cv2.waitKey(1) & 0xFF

    if key == ord("s"):  # Save frame if "s" is pressed
        frame_name = os.path.join(SAVE_DIR, f"frame_{len(os.listdir(SAVE_DIR)) + 1}.jpg")
        cv2.imwrite(frame_name, frame)
        print(f"Frame saved: {frame_name}")
    elif key == ord("q"):  # Quit if "q" is pressed
        break

cap.release()
cv2.destroyAllWindows()

# Process saved images
saved_images = [os.path.join(SAVE_DIR, img) for img in os.listdir(SAVE_DIR) if img.endswith(".jpg")]

if not saved_images:
    print("No frames saved to process.")
else:
    print("Processing saved frames...")
    results = model(saved_images)  # Run inference on saved images

    # Initialize a variable to store detected class names and costs
    detected_classes_with_costs = []

    # Process results
    for result in results:
        if result.boxes:
            class_names_and_costs = [
                {
                    "class_name": model.names[int(box.cls)],
                    "cost": class_costs.get(model.names[int(box.cls)], "N/A"),  # Default to "N/A" if no cost is specified
                }
                for box in result.boxes
            ]
        else:
            class_names_and_costs = []

        detected_classes_with_costs.append(class_names_and_costs)
        result.show()  # Display result
        result.save()  # Save visualized result

    print("********************************")
    total_cost = 0
    for image_idx, detections in enumerate(detected_classes_with_costs):
        print(f"Image {image_idx + 1}:")
        for detection in detections:
            print(f"Class: {detection['class_name']}, Cost: {detection['cost']}")
            if isinstance(detection["cost"], int):  # Ensure cost is numeric
                total_cost += detection["cost"]

    print("********************************")
    print(f"Total Cost: {total_cost}")
    print("********************************")

Press 's' to save frame, 'q' to quit.
Frame saved: E:\swift\ultralytics\results\frames\frame_1.jpg
Processing saved frames...

0: 480x640 1 soap, 106.5ms
Speed: 5.0ms preprocess, 106.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
********************************
Image 1:
Class: soap, Cost: 50
********************************
Total Cost: 50
********************************
